In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
# from sklearn_pandas import DataFrameMapper

In [ ]:
import torch
import torchtuples as tt

In [ ]:
from pycox.datasets import metabric
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

In [ ]:
np.random.seed(1234)
_ = torch.manual_seed(123)

In [ ]:
data = pd.read_csv("../clean_data/nafl/combined.large.nafl.csv")
Y = data[['StudyID', 'DaysUntilFirstProgression']]
# X = data.drop(columns='DaysUntilFirstProgression')

X = data.set_index('StudyID')
Y = Y.set_index('StudyID')

In [ ]:
Y = data[['StudyID', 'DaysUntilFirstProgression']]
# X = data.drop(columns='DaysUntilFirstProgression')

X = data.set_index('StudyID')
Y = Y.set_index('StudyID')

In [ ]:
# convert data to tensors
X_numpy = X.values.astype(np.float32) # turn into a numpy array
X_torch = torch.from_numpy(X_numpy)

Y_numpy = Y.values.astype(np.float32) # turn into a numpy array
Y_torch = torch.from_numpy(Y_numpy)

In [ ]:
# check if GPU is enabled
device = "cuda" if torch.cuda.is_available() else "cpu" # need to define device since python can use both cpu and gpu
print(f"Using {device} device")
print(f"Shape of X: {X.shape}. Shape of Y: {Y.shape}.")

In [ ]:
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X_torch, Y_torch, test_size=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

In [ ]:
in_features = X_train.shape[1]
num_nodes = [32, 32]
out_features = 1
batch_norm = True
dropout = 0.1
output_bias = False

net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
                              dropout, output_bias=output_bias)

In [ ]:
get_target = lambda df: (df['DaysUntilFirstProgression'].values, df['Outcome'].values.astype(int))
y_train = get_target(X_train)
y_val = get_target(X_val)
durations_test, events_test = get_target(X_test)
val = X_val, y_val

In [ ]:
X_train.astype(int)

In [ ]:
for array in y_train:
    array = array.astype('float32')

In [ ]:
y_train_new = (y_train[0].astype('float32'), y_train[1].astype('float32'))

In [ ]:
X_train = X_train.astype('float32')

In [ ]:
y_train_new[0].dtype

In [ ]:
y_train_new

In [ ]:
X_train['Lab_10466-1'].dtype

In [ ]:
for col in X_train.columns:
    if X_train[col].dtype != 'float32':
        print(f'This col is not float32: {col}')

In [ ]:
model = CoxPH(net, tt.optim.Adam)
batch_size = 256
lrfinder = model.lr_finder(X_train, y_train_new, batch_size, tolerance=10)
_ = lrfinder.plot()

In [ ]:
for child in net.named_parameters():
    print(child[1].dtype)